# Let's Play With the Speed of Sound
We're using the `PICMUS_carotid_cross.uff` dataset, which requires a good amount of computations. We should expect things to use a lot of memory and to run a bit slower than if using scan line imaging datasets.

We're using `"JAX"` as the backend.

In [ ]:
import jax
from pyuff_ustb import Uff

from vbeam.beamformers import get_beamformer
from vbeam.data_importers import import_pyuff
from vbeam.fastmath import backend_manager
from vbeam.util.download import cached_download

backend_manager.active_backend = "jax"

data_url = "http://www.ustb.no/datasets/PICMUS_carotid_cross.uff"
uff = Uff(cached_download(data_url))
channel_data = uff.read("/channel_data")
scan = uff.read("/scan")

imported_data = import_pyuff(channel_data, scan, frames=0)
jitted_beamformer = jax.jit(get_beamformer(imported_data))

# How Big is this Dataset?
`PICMUS_carotid_cross.uff` is pretty large. The image size is 387x609. Each pixel is imaged by each of the 128 receivers before being combined into an image. There are 75 transmits, aka images. There is however, only one frame

`387*609*128*75 = 2.26E+09` pixel calculations.

In [ ]:
x_axis_size = imported_data.scan.x.size
z_axis_size = imported_data.scan.z.size
num_frames = imported_data.signal.shape[0]
num_transmits = imported_data.signal.shape[1]
num_receivers = imported_data.signal.shape[2]
print((num_frames, num_transmits, num_receivers, x_axis_size, z_axis_size))
print(f"{num_frames*num_transmits*num_receivers*x_axis_size * z_axis_size:.2E} pixel calculations")


# Random Speed of Sound Samples
Let's generate random 20x20 speed of sound samples between 500 and 2000, beamform using that speed of sound, and save the result. Let's do it 10 times!

In [ ]:
import numpy as np
from vbeam.speed_of_sound import HeterogeneousSpeedOfSound
from vbeam.interpolation import FastInterpLinspace
import time

min_x, max_x, min_z, max_z = (
    imported_data.scan.x[0],
    imported_data.scan.x[-1],
    imported_data.scan.z[0],
    imported_data.scan.z[-1],
)

size = 20
x_axis = FastInterpLinspace(min_x, (max_x - min_x) / size, size)
z_axis = FastInterpLinspace(min_z, (max_z - min_z) / size, size)
n_samples = int(np.sqrt(2 * size**2))

result_samples = []
total_time_beamforming = 0
for i in range(10):
    sos_values = np.random.uniform(1400, 1700, (size, size))
    sos = HeterogeneousSpeedOfSound(sos_values, x_axis, z_axis, n_samples)

    start_time = time.perf_counter()
    result = jitted_beamformer(speed_of_sound=sos).block_until_ready()
    elapsed_time = time.perf_counter() - start_time
    total_time_beamforming += elapsed_time

    result_samples.append((result, sos_values))

f"{total_time_beamforming*100:.2f} milliseconds on average (including compilation)"


In [ ]:
import matplotlib.pyplot as plt

result, sos_values = result_samples[0]
fig, ax = plt.subplots(ncols=2)
fig.suptitle("What's wrong, doctor?")
sos_im = ax[0].imshow(sos_values.T, aspect="auto")
ax[0].set_title("Speed of sound")
beamformed_im = ax[1].imshow(result.T, aspect="auto")
ax[1].set_title("Beamformed image")
fig.tight_layout()

# Plotting the Results
Let's plot the results and save them as a GIF.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter


def make_animation(result_samples):
    result, sos_values = result_samples[0]

    fig, ax = plt.subplots(ncols=2)
    fig.suptitle("What's wrong, doctor?")
    sos_im = ax[0].imshow(sos_values.T, aspect="auto")
    ax[0].set_title("Speed of sound")
    beamformed_im = ax[1].imshow(result.T, aspect="auto")
    ax[1].set_title("Beamformed image")
    fig.tight_layout()

    def animate(i):
        result, sos_values = result_samples[i]
        beamformed_im.set_array(result.T)
        sos_im.set_array(sos_values.T)
        return (beamformed_im, sos_im)

    # call the animator.  blit=True means only re-draw the parts that have changed.
    return FuncAnimation(fig, animate, frames=len(result_samples) - 1, blit=True)


make_animation(result_samples).save("temp.gif", writer=PillowWriter(fps=6))


# Waving With the Waves
Here's a bonus animation!

In [ ]:
result_samples = []
for phase in np.linspace(0, np.pi * 2, 20):
    sos_values = (np.sin(np.arange(size) / size * np.pi - phase) + 2) * 1000
    sos_values = np.tile(sos_values, (size, 1)).T
    sos = HeterogeneousSpeedOfSound(sos_values, x_axis, z_axis, n_samples)
    result_samples.append((jitted_beamformer(speed_of_sound=sos), sos_values))

make_animation(result_samples).save("temp2.gif", writer=PillowWriter(fps=12))
